In [484]:
import pandas as pd
from dtaidistance import dtw_ndim
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import time
import seaborn as sns
import pyreadr
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram
# https://heartbeat.comet.ml/how-to-evaluate-clustering-based-models-in-python-503343816db2
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold, ShuffleSplit, RandomizedSearchCV, GridSearchCV, cross_val_score

##  Classification

In [558]:
# meta_num = pd.read_csv('filename')
labs = meta_num[['pco2','lact','ph_min','gluc_max',
             'inr','ptt','d_dimer','plts',
             'alt', 'ast','bili', 'creat', 'bun' ,
             'wbc_max','bands','alc']]

# make labels in to separate np array
label_orig = np.array(meta_num['cluster_label_orig'].tolist())
label_v2 = np.array(meta_num['cluster_label_v2'].tolist())
label_pen2 = np.array(meta_num['cluster_label_pen2'].tolist())

In [559]:
# scale each fature
scaler = StandardScaler()
labs_scaled = scaler.fit_transform(labs)

In [485]:
# comparing cross validation results on diff labels
hgb = HistGradientBoostingClassifier()
cv = ShuffleSplit(n_splits=5, test_size=0.2)
scores_hgb = cross_validate(hgb, labs_scaled, label_orig.ravel(), cv=cv, scoring=('accuracy','precision','recall','roc_auc'),return_train_score=True)

hgb = HistGradientBoostingClassifier()
scores_hgb_v2 = cross_validate(hgb, labs_scaled, label_v2.ravel(), cv=cv, scoring=('accuracy','precision','recall','roc_auc'),return_train_score=True)

hgb = HistGradientBoostingClassifier()
scores_hgb_pen2 = cross_validate(hgb, labs_scaled, label_pen2.ravel(), cv=cv, scoring=('accuracy','precision','recall','roc_auc'),return_train_score=True)

In [486]:
# print model output for original label
print ('train accuracy', scores_hgb['train_accuracy'].mean())
print ('train precision', scores_hgb['train_precision'].mean())
print ('train recall', scores_hgb['train_recall'].mean())
print ('train roc_auc', scores_hgb['train_roc_auc'].mean())
print ('test accuracy', scores_hgb['test_accuracy'].mean())
print ('test precision', scores_hgb['test_precision'].mean())
print ('test recall', scores_hgb['test_recall'].mean())
print ('test roc_auc', scores_hgb['test_roc_auc'].mean())

train accuracy 0.7679500926016181
train precision 0.7598408775308688
train recall 0.9752093555021307
train roc_auc 0.7975746912389609
test accuracy 0.7122027290448344
test precision 0.7249288128850352
test recall 0.9436462791777384
test roc_auc 0.6402245430891982


In [561]:
# print model output for data v2 penalty = 5
print ('train accuracy', scores_hgb_v2['train_accuracy'].mean())
print ('train precision', scores_hgb_v2['train_precision'].mean())
print ('train recall', scores_hgb_v2['train_recall'].mean())
print ('train roc_auc', scores_hgb_v2['train_roc_auc'].mean())
print ('test accuracy', scores_hgb_v2['test_accuracy'].mean())
print ('test precision', scores_hgb_v2['test_precision'].mean())
print ('test recall', scores_hgb_v2['test_recall'].mean())
print ('test roc_auc', scores_hgb_v2['test_roc_auc'].mean())

train accuracy 0.885290964031582
train precision 0.8929145847218971
train recall 0.15680969584395005
train roc_auc 0.8466927171378584
test accuracy 0.8707212475633529
test precision 0.544733455259771
test recall 0.06401160167846282
test roc_auc 0.6790992404943975


In [562]:
# print model output for data v2 penalty = 2
print ('train accuracy', scores_hgb_pen2['train_accuracy'].mean())
print ('train precision', scores_hgb_pen2['train_precision'].mean())
print ('train recall', scores_hgb_pen2['train_recall'].mean())
print ('train roc_auc', scores_hgb_pen2['train_roc_auc'].mean())
print ('test accuracy', scores_hgb_pen2['test_accuracy'].mean())
print ('test precision', scores_hgb_pen2['test_precision'].mean())
print ('test recall', scores_hgb_pen2['test_recall'].mean())
print ('test roc_auc', scores_hgb_pen2['test_roc_auc'].mean())

train accuracy 0.8756603957500732
train precision 0.9378429207796637
train recall 0.10894872903187425
train roc_auc 0.83471320523567
test accuracy 0.8617543859649123
test precision 0.43793795338786856
test recall 0.034577294558604324
test roc_auc 0.6616346161176383
